# The Problem

Suppose we have a coin that lands heads with probability $p$ and tails with probability $q = p - 1$, and which has no memory between flips (meaning successive flips are stochastically independent). What are the chances of seeing two consecutive heads in n coin flips?

## The Sample Space

Representing the possible outcomes of three tosses as a tree diagram, we have:

![tree diagram for three tosses of a coin](tree0.png)

Each path has a weight equal to its probability. Since successive trials are stochastically independent, each branch leading to an H has a weight of p and each branch leading to a T has a weight of q.

Although we're trying to determine the probability of two consecutive heads in n flips, it's arguably easier to characterize its complement, the probability of *not* getting two consecutive heads in n flips. Suppose that, as of the kth flip, we have not yet seen two consecutive heads. The kth flip can be either a head or a tail. Suppose it's a head; in this case the the k-1st flip must have been a tail. So the first k flips must end either in TH or T, of probability $pq$ and $q$, respectively. Denoting the probability of not getting two consecutive heads as of flip k as $a_k$, we then have $a_k = pqa_{k-2} + qa_{k-1}$.

To see this in the tree diagram for k = 2:

![paths without two consecutive heads on toss 2](tree1.png)

And k = 3:

![paths without two consecutive heads on toss 3](tree2.png)

## Solving for $a_i$

This recursive formulation is useful and, indeed, one way to compute the probability for two consecutive heads in n flips is to perform the computation recursively:

In [1]:
def no_two_consecutive_heads_recursive(n, p, q):
    a_k_2, a_k_1 = 1, 1
    a_k = None
    for k in range(2, n + 1):
        a_k = a_k_2 * p * q + a_k_1 * q
        a_k_2 = a_k_1
        a_k_1 = a_k
    return a_k

def two_consecutive_heads_recursive(n, p, q):
    return 1 - no_two_consecutive_heads_recursive(n, p, q)

Trying this out for a coin that lands heads with p = 1/4 and with n = 5, we have:

In [2]:
print(two_consecutive_heads_recursive(5, 0.25, 0.75))

0.2001953125


One way to check this result is by counting:

In [3]:
def weighted_sequences(n, r, s):
    """
    Yields all strings consisting of {H,T} with counts consistent with 
    P(H) = r/(r + s) and P(T) = s/(r + s).
    """
    strings = [""]
    while len(strings) > 0:
        string = strings.pop()
        if len(string) == n:
            yield string
        else:
            for i in range(r):
                strings.append(string + "H")
            for i in range(s):
                strings.append(string + "T")

def two_consecutive_heads_counting(n, r, s):
    num_total = 0
    num_two_consecutive_heads = 0
    for sequence in weighted_sequences(n, r, s):
        num_total += 1
        if "HH" in sequence:
            num_two_consecutive_heads += 1
    return num_two_consecutive_heads / num_total

print(two_consecutive_heads_counting(5, 1, 3))

0.2001953125


This verifies that the recursive expression for $a_i$ is correct, but can we come up with a closed-form expression for $a_i$?
## A closed-form expression for $a_i$ using generating functions
Using generating functions, we can come up with a closed form expression for $a_i$.

\begin{alignat}{10}
g(x) & = 1 & + && x & + & a_2x^2 & + & a_3x^3 & + ...\\
qxg(x) & = &&& qx & + & qx^2 & + & qa_2x^3 & + ...\\
pqx^2g(x) & = &&&&& pqx^2 & + & pqx^3 & + ...\\
\end{alignat}

Solving for g(x) by subtracting the latter two expressions from the first, we obtain

\begin{equation*}
g(x) = \frac{1 + (1 - q)x}{1 - qx - pqx^2}
\end{equation*}

Define $\alpha$ and $\beta$ as

\begin{align*}
\alpha & = \frac{q - \sqrt{q(4p + q)}}{2}\\
\beta & = \frac{q + \sqrt{q(4p + q)}}{2}
\end{align*}

A little arithmetic later and we have:

\begin{equation*}
g(x) = \frac{1}{\sqrt{q(4p + q)}}\left(\frac{\beta - 1}{1 - {\alpha}x} - \frac{\alpha - 1}{1 - {\beta}x}\right)
\end{equation*}

Equating coefficients for $1/(1-sx)$, we have a closed-form expression for the nth term:

\begin{equation*}
a_n = \frac{1}{\sqrt{q(4p + q)}}\left((\beta - 1)\alpha^n - (\alpha - 1)\beta^n\right)
\end{equation*}

As a form of verification, take a look at the result of this computation for our previous values of n and p:

In [4]:
from math import sqrt, pow

def no_two_consecutive_heads_closed(n, p, q):
    s = sqrt(q * (4 * p + q))
    alpha = (q - s) / 2
    beta = (q + s) / 2
    return ((beta - 1) * pow(alpha, n) - (alpha - 1) * pow(beta, n)) / s

For a coin that lands heads with p = 1/4 and with n = 5, we see this returns a number that matches the other two methods:

In [5]:
1 - no_two_consecutive_heads_closed(5, 0.25, 0.75)

0.20019531250000022

## Computing $a_i$ using a Markov matrix

Suppose we have a process that has 3 possible states, and that transitions from state $j$ to state $i$ with probability $a_{ij}$, and that the probability of each transition depends upon current state and no other aspect of history. Suppose that, over the course of k state transitions, the process transitions from state $j$ to state $i$ with probability $b_{ij}$. What is the probability $c_{ij}$ it will transition from state $j$ to state $i$ in k + 1 state transitions? (Go ahead and try to work this out, answer follows).

We know that, in the first state transition, the process will go from $j$ to $l$ with probability $a_{lj}$. We then know that, for each target state $l$, the process will go to $i$ over the next k transitions with probability $b_{il}$. Since there are three possible states, we have:

\begin{equation*}
c_{ij} = \sum_{l=1}^{3} b_{il}a_{lj}
\end{equation*}

Hopefully this looks like the product of matrix B, which consists of entries $b_{ij}$ and matrix A, which consists of entries $a_{ij}$:

\begin{equation*}
\begin{pmatrix}
b_{11} & b_{12} & b_{13} \\
b_{21} & b_{22} & b_{23} \\
b_{31} & b_{32} & b_{33}
\end{pmatrix} \begin{pmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33}
\end{pmatrix} = \begin{pmatrix}
c_{11} & c_{12} & c_{13} \\
c_{21} & c_{22} & c_{23} \\
c_{31} & c_{32} & c_{33}
\end{pmatrix}
\end{equation*}

Using induction, we can restate this as: the probability of going from state i to state j in k + 1 state transitions is exactly equaly to $A$ multiplied by itself k+1 times:

\begin{equation*}
\begin{pmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33}
\end{pmatrix}^{k+1}
\end{equation*}

I wish I were the person who invented this formalism, but [Andrey Markov first published on the subject in 1906](https://en.wikipedia.org/wiki/Stochastic_matrix).

How does this help us? Define n tosses of a coin as a process that has three distinct states:

1. Toss n came up tails, and we have not yet seen two consecutive heads.
2. Toss n came up heads, and we have not yet seen two consecutive heads.
3. As of toss n (inclusive), we have seen two consecutive heads.

The transition matrix corresponding to these three states:

\begin{equation*}
\begin{pmatrix}
q & q & 0 \\
p & 0 & 0 \\
0 & p & 1
\end{pmatrix}
\end{equation*}

Quick explanation:

1. Suppose we are in state 1. We remain in state 1 by landing on tails, which has probability $q$, and we move to state 2 by landing on heads, which has probability $p$. There is no way to get to state 3.
2. Suppose we are in state 2. We can revert to state 1 by landing on tails, which has probability $q$, and we can move to state 3 by landing on heads. There is no way to get from state 2 to 2.
3. Suppose we are in state 3. Then we remain in state 3 forever; the only available state transition is from 3 to 3.

Checking this computationally using our previous parameters:

In [6]:
import numpy as np

def two_consecutive_heads_matrix(n, p, q):
    A = np.matrix([[q, q, 0], [p, 0, 0], [0, p, 1]])
    return (A ** n).item((2, 0))

print(two_consecutive_heads_matrix(5, 0.25, 0.75))

0.2001953125


## k consecutive heads in n flips

### Markov matrix method

Generalizing the previous result, we could define n tosses of a coin as a process that has k+1 distinct states:

&nbsp;&nbsp;1\. Toss n came up tails, and we have not yet seen k consecutive heads<br/>
&nbsp;&nbsp;2\. Toss n came up heads, and we have not yet seen k consecutive heads<br/>
&nbsp;&nbsp;...<br/>
&nbsp;&nbsp;k\. Tosses n - k + 2 to n (inclusive) came up heads, and we have not yet seen k consecutive heads<br/>
&nbsp;&nbsp;k+1\. As of toss n (inclusive), we have seen k consecutive heads

Computing this using Python:

In [10]:
def k_consecutive_heads_matrix(n, k, p, q):
    matrix = [[q] * k + [0]]
    for i in range(k):
        matrix.append([0] * i + [p] + [0] * (k - i))
    matrix[-1][-1] = 1
    return (np.matrix(matrix) ** n).item((k, 0))

print(k_consecutive_heads_matrix(5, 3, 0.25, 0.75))

0.0390625


Verifying this result by simple counting:

In [7]:
def k_consecutive_heads_counting(n, k, r, s):
    k_heads = "H" * k
    num_total = 0
    num_k_consecutive_heads = 0
    for sequence in weighted_sequences(n, r, s):
        num_total += 1
        if k_heads in sequence:
            num_k_consecutive_heads += 1
    return num_k_consecutive_heads / num_total

In [8]:
print(k_consecutive_heads_counting(5, 3, 1, 3))

0.0390625


Numpy is likely using a matrix multiplication method that is sub-$O(k^3)$ for a k-by-k square matrix. Because it squares matrices as a shortcut when taking a matrix to a given power, the time complexity of this method is $O(k^3log(n))$.

### Dynamic programming method

Reflecting on [our original method](#Solving-for-%24a_i%24) to recursively solve for the probability of not seeing two consecutive heads as of i flips, $a_i$, the method can be extended to recursively solve for the probability of not seeing k consecutive heads as of i flips:

\begin{equation*}
a_i = \sum_{j=1}^{k} qp^{j-1}a_{i-j}
\end{equation*}

In Python: